In [1]:
import numpy as np
import pandas as pd

from svm_proxy import *

In [ ]:
class test_proxy:

    def __init__(self, proxy, true_weights, true_intercept, logs = 'all'):
        self.logs = logs
        self.true_weights = true_weights
        self.true_intercept = true_intercept
        self.proxy = proxy
        self.proxy_intercept 

        # projection approach


    def steal(self, attacker, max_queries): 

        for query in range(max_queries):
            # attacker fits to all available queries


        